<a href="https://colab.research.google.com/github/saez1007/PPO-Tensorflow-2.0/blob/master/PPO_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/saez1007/PPO-Tensorflow-2.0.git
!pip install -e "PPO-Tensorflow-2.0"

Cloning into 'PPO-Tensorflow-2.0'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 142 (delta 30), reused 127 (delta 20), pack-reused 0
Receiving objects: 100% (142/142), 1.33 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Obtaining file:///content/PPO-Tensorflow-2.0
     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 81kB 6.3MB/s 
     |████████████████████████████████| 327kB 20.1MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 215kB 19.1MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 747kB 15.9MB/s 
  Running 

In [0]:
# Kernel needs to be restarted before detecting the package
import gym
import pandas as pd
import numpy as np
from gym import spaces
from sklearn import preprocessing
from datetime import datetime
import tensorflow as tf
import os
from rl_tf_2.algorithms.ppo import CategoricalModel, GaussianModel, Roller, PolicyCombinedLoss, network_builder, Params, mlp
from rl_tf_2.environment import create_batched_env
from rl_tf_2.utils import Logger, log

In [0]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/Kraken_BTCUSD_d.csv", skiprows= 2, names = ['date', 'symbol', 'open', 'high', 'low', 'close', 'vol_btc', 'vol']).sort_index(ascending=False)

In [0]:
class BitcoinTradingEnv(gym.Env):
    """A Bitcoin trading environment for OpenAI gym"""
    metadata = {'render.modes': ['live', 'file', 'none']}
    scaler = preprocessing.MinMaxScaler()
    viewer = None

    def __init__(self,
                 df,
                 lookback_window_size=0,
                 commission=0.00075,
                 initial_balance=10000,
                 max_session = 300,
                 serial=False
                 ):
        super(BitcoinTradingEnv, self).__init__()
        self.env_info = EnvInfo
        self.df = df.dropna().reset_index()
        self.lookback_window_size = lookback_window_size
        self.initial_balance = 0
        # self.commission = commission
        self.serial = serial
        self.max_session = max_session
        # Actions of the format Buy 1/10, Sell 3/10, Hold, etc.
        self.action_space = spaces.Discrete(3)
        # Observes the OHCLV values
        self.observation_space = spaces.Box(low=0, high=1, shape=(5
                                                                  ,self.lookback_window_size + 1
                                                                  ), dtype=np.float16)

    def reset(self):
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.btc_held = 0
        self._reset_session()
        self.account_history = np.repeat([
        [0],
        [0],
        [0],
        [0],
        [0]
        ], self.lookback_window_size + 1, axis=1)
        self.trades = []
        return self._next_observation()[0], {}, {}, {}

    def _reset_session(self):
        self.current_step = np.random.randint(0, len(self.df) - self.max_session)
        if self.serial:
            self.steps_left = len(self.df) - self.lookback_window_size - 1
            self.frame_start = self.lookback_window_size
        else:
            self.steps_left = np.random.randint(1, self.max_session)
            self.frame_start = np.random.randint(
                self.lookback_window_size, len(self.df) - self.steps_left)
            self.active_df = self.df[self.frame_start -
                self.lookback_window_size:self.frame_start + self.steps_left]

    def _next_observation(self):
        end = self.current_step + self.lookback_window_size + 1
        obs = np.array([
        self.df['open'].values[self.current_step:end],
        self.df['high'].values[self.current_step:end],
        self.df['low'].values[self.current_step:end],
        self.df['close'].values[self.current_step:end],
        self.df['vol'].values[self.current_step:end],
        ])
        new_price = self.df['close'].values[self.current_step:][0]
        scaled_history = self.scaler.fit_transform(self.account_history)
        # obs = np.append(obs, scaled_history[:, -(self.lookback_window_size
        #                                                  + 1):], axis=0)
        return obs, new_price

    def step(self, action):
        current_price = self._get_current_price()
        self._take_action(action, current_price)
        self.steps_left -= 1
        self.current_step += 1
        if self.steps_left == 0:
            self.balance += self.btc_held * current_price
            self.btc_held = 0
            self._reset_session()
        obs, new_price = self._next_observation()
        # Rewards given depending on the sign of the stock price
        import pdb; pdb.set_trace()
        if action == 0:
            reward = 1 if new_price == current_price else 0
        elif action == 1:
            reward = 1 if new_price < current_price else 0
        elif action == 2:
            reward = 1 if new_price > current_price else 0
        done = False
        return obs, {}, reward, done, {}

    def _get_current_price(self):

        return self.df['close'].values[self.current_step:][0]

    def _take_action(self, action, current_price):
        action_type = action
        # amount = action[1] / 10
        btc_bought = 0
        btc_sold = 0
        cost = 0
        sales = 0
        # if action_type < 1:
        #     btc_bought = self.balance / current_price * amount
        #     cost = btc_bought * current_price * (1 + self.commission)
        #     self.btc_held += btc_bought
        #     self.balance -= cost
        # elif action_type < 2:
        #     btc_sold = self.btc_held * amount
        #     sales = btc_sold * current_price * (1 - self.commission)
        #     self.btc_held -= btc_sold
        #     self.balance += sales

        # if btc_sold > 0 or btc_bought > 0:
        # self.trades.append({
        #   'step': self.frame_start+self.current_step,
        #   'amount': btc_sold if btc_sold > 0 else btc_bought,
        #   'total': sales if btc_sold > 0 else cost,
        #   'type': "sell" if btc_sold > 0 else "buy"
        # })
        # self.net_worth = self.balance + self.btc_held * current_price
        self.account_history = np.append(self.account_history, [
        [self.net_worth],
        [btc_bought],
        [cost],
        [btc_sold],
        [sales]
        ], axis=1)

    # def render(self, mode='human', **kwargs):
    #   if mode == 'human':
    #     if self.viewer == None:
    #       self.viewer = BitcoinTradingGraph(self.df,
    #                                         kwargs.get('title', None))
    #     self.viewer.render(self.frame_start + self.current_step,
    #                        self.net_worth,
    #                        self.trades,
    #                        window_size=self.lookback_window_size)

class EnvInfo():
    env_name: str
    academy_name: str
    num_agents: int
    is_discrete: bool
    is_visual: bool
    is_vector: bool
    is_frame_stacking: bool
    stack_size: int
    shapes: dict
    act_size: int
    



env_test = BitcoinTradingEnv(data)

In [18]:
env_test.action_space.n

3

In [0]:
# Network without envinfo
def simple_actor_critic( hidden_sizes= (32, 32), activation= 'relu', activation_output= None, 
                         kernel_initalizer= 'glorot_uniform', name= 'simple_actor_critic', env_info= EnvInfo):

    _actor = mlp(   hidden_sizes= hidden_sizes, output_size= env_info.action_space.n, activation= activation, 
                    activation_output= activation_output, name= name, kernel_initalizer= kernel_initalizer)
    
    _critic = mlp(  hidden_sizes= hidden_sizes, output_size= 1, activation= activation, 
                    activation_output= activation_output, name= name, kernel_initalizer= kernel_initalizer)

    log('Model Summary: ' + name)

    _actor.build(input_shape = (None,) + env_info.observation_space.shape)
    _actor.summary()

    _critic.build(input_shape = (None,) + env_info.observation_space.shape)
    _critic.summary()

    def forward(inp= None):
        logits = _actor(inp['vec_obs'])
        values = _critic(inp['vec_obs'])
        return logits, values

    return {"forward": forward, "trainable_networks": [_actor, _critic]}


def vis_vec_actor_critic (  hidden_sizes= (32, 32), activation= 'relu', activation_output= None, 
                            kernel_initalizer= 'glorot_uniform', name= 'vis_vec_actor_critic', env_info = env_test):

    cnn, out_units = cnn_simple()

    _mlp_actor = mlp(   hidden_sizes= hidden_sizes, output_size= env_info.action_space, activation= activation, 
                        activation_output= activation_output, name= name, kernel_initalizer= kernel_initalizer)

    _mlp_critic = mlp(  hidden_sizes= hidden_sizes, output_size= 1, activation= activation, 
                        activation_output= activation_output, name= name, kernel_initalizer= kernel_initalizer)
    
    log('Model Summary: ' + name)

    cnn.build(input_shape = (None,) + env_info.shapes['vis'])
    cnn.summary()
    _mlp_actor.build(input_shape= (None, env_test.observation_space + out_units))
    _mlp_actor.summary()
    _mlp_critic.build(input_shape= (None, env_info.observation_space + out_units))
    _mlp_critic.summary()

    def forward(inp= None):
        out_cnn = cnn(inp['vis_obs'])                               # Convolutional Network for visuals 
        # out_vec_mlp = _mlp_vec(inp['vec_obs])                     # Put vec_obs thorugh Neural Network if to much features 
        mixed = tf.concat([out_cnn, inp['vec_obs']], -1)            # Concatenate cnn and vec_obs or out_vec_mlp
        # out_mixer_mlp = _mlp_mixer(mixed)                         # state mixer with Neural Network if needed
        logits = _mlp_actor(mixed)                                  # Feed with raw mixed or with out_mixer_mlp
        values = _mlp_critic(mixed)
        return logits, values

    return {"forward": forward, "trainable_networks": [cnn, _mlp_actor, _mlp_critic]}


class CategoricalModel_2(tf.keras.Model):
    """
        Categorical Model for Discrete Action Spaces
        --------------------------------------------
            Input:
                Network with foward pass and EnvInfo Object
            Returns:   
                call:                   logits, values from Neural Network 
                                        with defined forward pass
                get-action-lop-value:   logp, action, value 
                                        (action drawn from Random Categ. Dist)
                logp:                   Log probability for action x
                entropy:                Entropy Term from logits
    """
    def __init__(self, network= None, env_info= env_test):
        super().__init__('CategoricalModel')

        self.env_info = env_info
        self.act_size = self.env_info.action_space.n                              # Number of possible actio
        self.forward = network['forward']                                   # Get feed forward chain
        self.all_networks = network['trainable_networks']                   # Get all trainable networks
  
    def pd(self, logits):
        return tf.squeeze(tf.random.categorical(logits, 1), axis=-1)        # Draw from Random Categorical Distribution

    # @tf.function
    def call(self, inputs):
        return self.forward(inp = inputs) 
        
    def get_action_logp_value(self, obs):
        """
            Returns:
            logits --> Last layer of Neural Network without activation function \n
            logp --> SOFTMAX of logits which squashes logits between 0 .. 1 and returns log probabilities \n
            actions --> drawn from normal distribution
        """
        logits, values = self.predict(obs)
        import pdb; pdb.set_trace()                                  # Returns np arrays on predict | Input: np array or tensor or list
        actions = self.pd(logits)
        logp_t = self.logp(logits, actions) 
        return np.squeeze(actions), np.squeeze(logp_t), np.squeeze(values) 

    def logp(self, logits, action):
        """
            Returns:
            
            logp based on the action drawn from prob-distribution \n
            indexes in the logp_all with one_hot
        """
        logp_all = tf.nn.log_softmax(logits)
        one_hot = tf.one_hot(action, depth= self.act_size)
        logp = tf.reduce_sum( one_hot * logp_all, axis= -1)
        return logp
        
    def entropy(self, logits= None):
        """
            Entropy term for more randomness which means more exploration \n
            Based on OpenAI Baselines implementation
        """
        a0 = logits - tf.reduce_max(logits, axis= -1, keepdims=True)
        exp_a0 = tf.exp(a0)
        z0 = tf.reduce_sum(exp_a0, axis= -1, keepdims=True)
        p0 = exp_a0 / z0
        entropy = tf.reduce_sum(p0 * (tf.math.log(z0) - a0), axis= -1)
        return entropy



In [0]:

class Roller:

    def __init__(self, batched_env, model, num_steps, gamma= 0.99, lam= 0.97):

        self.batched_env = batched_env
        self.batched_env.env_info.is_vector = True
        self.batched_env.env_info.is_visual = False
        self.model = model
        self.num_steps = num_steps
        self.gamma = gamma
        self.lam = lam
        self._obs_vec = None
        self._obs_vis = None
        self._rews = None
        self._dones = None
        self._first_reset = False

    def reset(self):
        self._obs_vec, self._obs_vis, self._rews, self._dones = self.batched_env.reset()

    def rollout(self):

        if not self._first_reset:                       # self._obs is None: only reset on first time
            self.reset()
            self._first_reset = True
        
        vec_obses = []
        vis_obses = []
        rews = []
        dones = []
        actions = []
        values = []
        logp = []
        ep_rews = []
        ep_lens = []

        for step in range(self.num_steps):              # Run each env for num_steps and gather trajectory rollouts

            actions_t, logp_t, values_t = self.model.get_action_logp_value({"vec_obs": self._obs_vec, "vis_obs": self._obs_vis})
            import pdb; pdb.set_trace()
            vec_obses.append(self._obs_vec) if self.batched_env.env_info.is_vector else None
            vis_obses.append(self._obs_vis) if self.batched_env.env_info.is_visual else None
            dones.append(self._dones)
            actions.append(actions_t)
            values.append(values_t)
            logp.append(logp_t)

            self._obs_vec, self._obs_vis, self._rews, self._dones, infos = self.batched_env.step(actions_t)

            for info in infos:
                if info is not None:
                    ep_rews.append(info['ep_rew'])
                    ep_lens.append(info['ep_len'])

            rews.append(self._rews)                     # Saves obs(t), dones(t), actions(t), logp(t), values(t) , rews(t+1) in one cycle
                                                        # Saves rews(t+1) after env step with action(t)

        """
            End of for loop
            ---------------
            Get last Values for BOOTSTRAPING
        """
        actions_t, logp_t, values_t = self.model.get_action_logp_value({"vec_obs": self._obs_vec, "vis_obs": self._obs_vis})
        last_values = values_t                          # Bootstraping

        vec_obses = np.array(vec_obses, dtype= np.float32) if self.batched_env.env_info.is_vector else None
        vis_obses = np.array(vis_obses, dtype= np.float32) if self.batched_env.env_info.is_visual else None
        rews = np.array(rews, dtype=np.float32)
        dones = np.array(dones, dtype= np.bool)
        values = np.array(values, dtype=np.float32)
        logp = np.array(logp, dtype=np.float32)
        actions = np.array(actions, dtype=np.float32) if isinstance(self.model, GaussianModel) else np.array(actions, dtype=np.int32)
        
        """
            Discount / Bootstrap Values and calc Advantages
            -----------------------------------------------
        """
        returns = np.zeros_like(rews)
        advs = np.zeros_like(rews)
        last_gae_lam = 0

        for t in reversed(range(self.num_steps)):
            if t == self.num_steps - 1:
                next_non_terminal = 1.0 - self._dones
                next_values = last_values
            else:
                next_non_terminal = 1.0 - dones[t + 1]
                next_values = values[t + 1]
            
            delta = rews[t] + self.gamma * next_values * next_non_terminal - values[t]
            advs[t] = last_gae_lam = delta + self.gamma * self.lam * next_non_terminal * last_gae_lam
            
        returns = advs + values                         # ADV = RETURNS - VALUES
        advs = (advs - advs.mean()) / (advs.std())      # Normalize ADVs

        return self._flattened_rollout(vec_obses, vis_obses, rews, dones, actions, logp, values, advs, returns), \
               {'ep_rews': ep_rews, 'ep_lens': ep_lens}
    
    def _flattened_rollout(self, vec_obses, vis_obses, rews, dones, actions, logp, values, advs, returns):

        if self.batched_env.env_info.is_visual:         # Reshape visual obs --> flatten array
            d1, d2, d3, d4, d5 = vis_obses.shape
            vis_obses = vis_obses.reshape(d1 * d2, d3, d4, d5)
        
        if self.batched_env.env_info.is_vector:         # Reshape flatten vec obs
            vec_obses = vec_obses.reshape(-1, vec_obses.shape[-1])
        
        if isinstance(self.model, GaussianModel):
            actions = actions.reshape(-1, actions.shape[-1])
        else:
            actions = actions.flatten()

        return{     'vec_obses': vec_obses, 
                    'vis_obses': vis_obses,
                    'rews': rews.flatten(), 
                    'dones': dones.flatten(), 
                    'actions': actions,
                    'logp': logp.flatten(), 
                    'values': values.flatten(), 
                    'advs': advs.flatten(), 
                    'returns': returns.flatten()}

In [0]:
config = '/PPO-Tensorflow-2.0/__WORKING_DIRS__/__STANDARD__/__EXAMPLE__.yaml'

params = Params(working_dir= os.getcwd(), config_file= config)          # Get Configuration | HORIZON = Steps per epoch

tf.random.set_seed(params.env.seed)                                     # Set Random Seeds for np and tf
np.random.seed(params.env.seed)

env = BitcoinTradingEnv(data)    # Create Environment in multiprocessing mode
LOGGER = Logger('academy_name', os.getcwd(), config)         # Set Logger

# network = network_builder(params.trainer.nn_architecure) \
#     (hidden_sizes=params.policy.hidden_sizes, env_info=env.env_info)    # Build Neural Network with Forward Pass

network = simple_actor_critic(env_info = env_test)   # Our own version of the simple_actor_critic that fits with the environment

model = CategoricalModel_2
model = model(network=network, env_info=env_test)                   # Build Model for Discrete or Continuous Spaces

if params.trainer.load_model:
    log('Loading Model ...')
    model.load_weights(LOGGER.tf_model_path('model_weights'))           # Load model if load_model flag set to true

roller = Roller(env_test, model, params.trainer.steps_per_epoch,
                params.trainer.gamma, params.trainer.lam)               # Define Roller for genrating rollouts for training

ppo = PolicyCombinedLoss(model=model, num_envs=1)            # Define PPO Policy with combined loss

for epoch in range(params.trainer.epochs):                              # Main training loop for n epochs

    rollouts, infos = roller.rollout()                                  # Get Rollout and infos
    outs = ppo.update(rollouts)                                         # Push rollout in ppo and update policy accordingly

    LOGGER.store('Loss Pi', outs['pi_loss'])
    LOGGER.store('Loss V', outs['v_loss'])
    LOGGER.store('Loss Ent', outs['entropy_loss'])
    LOGGER.store('Appr Ent', outs['approx_ent'])
    LOGGER.store('KL DIV', outs['approx_kl'])

    for ep_rew, ep_len in zip(infos['ep_rews'], infos['ep_lens']):
        LOGGER.store('EP REW', ep_rew)
        LOGGER.store('EP LEN', ep_len)

    if (epoch % params.trainer.save_freq == 0) or (epoch == params.trainer.epochs - 1):
        log('Saving Model ...')
        model.save_weights( LOGGER.tf_model_path('model_weights'), 
                            save_format='tf')                           # Saving model-weights every n steps

    LOGGER.log_metrics(epoch)                                           # Push metrics to screen

env.close()                                           


-----------------------------------------------------------------
Model Summary: simple_actor_critic
-----------------------------------------------------------------

Model: "simple_actor_critic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_actor_critic (Dense)  multiple                  64        
_________________________________________________________________
simple_actor_critic (Dense)  multiple                  1056      
_________________________________________________________________
simple_actor_critic_output ( multiple                  99        
Total params: 1,219
Trainable params: 1,219
Non-trainable params: 0
_________________________________________________________________
Model: "simple_actor_critic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_actor_critic (Dense)  multiple        